In [1]:
import pandas as pd
import numpy as np

df_market = pd.read_csv('datasets/train.csv')
df_tweets = pd.read_csv('datasets/tweets_from2015_#Ibex35.csv')

In [2]:
df_market.shape

(6554, 8)

In [3]:

df_market.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Target
0,1994-01-03,3615.199951,3654.699951,3581.000000,3654.500000,3654.496338,0.0,0
1,1994-01-04,3654.500000,3675.500000,3625.100098,3630.300049,3630.296387,0.0,1
2,1994-01-05,3625.199951,3625.199951,3583.399902,3621.199951,3621.196289,0.0,1
3,1994-01-06,NaN,NaN,NaN,NaN,NaN,NaN,0
4,1994-01-07,3621.199951,3644.399902,3598.699951,3636.399902,3636.396240,0.0,1


In [4]:
df_tweets.shape

(9801, 3)

In [5]:
df_tweets.head()

,tweetDate,handle,text
0,Sat Apr 09 14:47:45 +0000 2022,abelac62,He hecho el repaso de todos los componentes de...
1,Thu Apr 07 19:14:36 +0000 2022,LluisPerarnau,Els projectes que han presentat les empreses d...
2,Mon Apr 04 16:48:45 +0000 2022,Pegaso121080,"Por si no lo has visto, o no lo encuentras en ..."
3,Tue Apr 05 07:23:16 +0000 2022,zonavalue,📈 #BOLSA: El #Ibex35 abre en 🟢 \n\n🇪🇸 #Ibex35 ...
4,Thu Mar 31 16:07:43 +0000 2022,EPeconomia,"El #Ibex35 retrocede un 0,4% en marzo y un 3,0..."


In [6]:
print(df_market.isnull().sum()) # amount of days without register

Date           0
Open         133
High         133
Low          133
Close        133
Adj Close    133
Volume       133
Target         0
dtype: int64


In [7]:
null_map = df_market.isnull()

def get_pos_no_null_forward(null_map, column, i, pos_forward):
    while null_map[column][i + pos_forward] == True:
        pos_forward += 1
    return pos_forward

def get_pos_no_null_backward(null_map, column, i, pos_backward):
    while null_map[column][i - pos_backward] == True:
        pos_backward += 1
    return pos_backward

columns = df_market.columns

for column in columns:
    if column != 'Date' and column != 'Target':
        for i in range(0, len(null_map)):
            pos_backward = 1
            pos_forward = 1
            if null_map[column][i] == True:
                pos_backward = get_pos_no_null_backward(null_map, column, i, pos_backward)
                pos_forward = get_pos_no_null_forward(null_map, column, i, pos_forward)
                df_market.at[i, column] = (df_market[column][i - pos_backward] + df_market[column][i + pos_forward]) / 2

In [8]:
print(df_market.isnull().sum())

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
Target       0
dtype: int64


In [9]:
print(df_market.isnull())
df_market.to_csv('datasets/clean.csv')

       Date   Open   High    Low  Close  Adj Close  Volume  Target
0     False  False  False  False  False      False   False   False
1     False  False  False  False  False      False   False   False
2     False  False  False  False  False      False   False   False
3     False  False  False  False  False      False   False   False
4     False  False  False  False  False      False   False   False
...     ...    ...    ...    ...    ...        ...     ...     ...
6549  False  False  False  False  False      False   False   False
6550  False  False  False  False  False      False   False   False
6551  False  False  False  False  False      False   False   False
6552  False  False  False  False  False      False   False   False
6553  False  False  False  False  False      False   False   False

[6554 rows x 8 columns]


In [15]:

x = pd.DataFrame()

x['HL'] = df_market['High'] - df_market['Low']
x['OC'] = df_market['Open'] - df_market['Close']
x['7MA'] = df_market['Close'].rolling(window=7, min_periods=1).mean()
x['14MA'] = df_market['Close'].rolling(window=14, min_periods=1).mean()
x['21MA'] = df_market['Close'].rolling(window=21,min_periods=1).mean()
x['STDV'] = df_market['Close'].rolling(window=7, min_periods=1).std()
x.at[0, 'STDV'] = x['STDV'][1]

print (x)



             HL         OC          7MA         14MA         21MA       STDV
0     73.699951 -39.300049  3654.500000  3654.500000  3654.500000  17.111949
1     50.399902  24.199951  3642.400024  3642.400024  3642.400024  17.111949
2     41.800049   4.000000  3635.333333  3635.333333  3635.333333  17.211153
3     43.750000  -5.599975  3633.699982  3633.699982  3633.699982  14.427539
4     45.699951 -15.199951  3634.239966  3634.239966  3634.239966  12.552821
...         ...        ...          ...          ...          ...        ...
6549  69.699218 -24.299804  9220.571289  9183.592704  9277.337983  64.470992
6550  89.899414   9.500000  9208.014230  9182.257045  9263.776088  52.981485
6551  92.000000  28.600586  9195.399972  9179.742745  9248.814174  42.416577
6552  81.600586  32.700195  9178.371373  9178.692732  9228.018927  60.482360
6553  61.100586 -37.000000  9166.757115  9181.571289  9208.361793  54.374824

[6554 rows x 6 columns]


In [47]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import RandomForestRegressor as RFR

rfc = RFC(random_state=42, n_jobs=-1, max_depth=5,
                                       n_estimators=100, oob_score=True)

predicted_y = rfc.fit(x, df_market['Target'])

0.5329569728410132


In [23]:
from sklearn.model_selection import GridSearchCV

rf = RFC(random_state=42, n_jobs=-1)

params = {
    'max_depth': [2,3,5,10,20],
    'min_samples_leaf': [5,10,20,50,100,200],
    'n_estimators': [10,25,30,50,100,200]
}

grid_search = GridSearchCV(estimator=rf,
                           param_grid=params,
                           cv = 4,
                           n_jobs=-1, verbose=1, scoring="accuracy")

grid_search.fit(x, df_market['Target'])
rf_best = grid_search.best_estimator_

Fitting 4 folds for each of 180 candidates, totalling 720 fits


In [37]:
grid_search.best_score_

0.5329569728410132


In [48]:
from sklearn.tree import plot_tree
import matplotlib as plt
from sklearn.model_selection import train_test_split

# plt.figure(figsize=(80,40))
# plot_tree(rf_best.estimators_[5], feature_names = x.columns,class_names=['0', "1"],filled=True);

rfr = RFR(oob_score=True)

X_train, X_test, y_train, y_test = train_test_split(x, df_market['Close'], train_size=0.7, random_state=42)

prediction = rfr.fit(X_train, y_train)


In [49]:
print (prediction.oob_score_)
print (rfr.score(X_test, y_test))

0.9974373627022483
0.9977127390034531
